In [74]:
from pymongo import MongoClient
import json
import folium

查询操作

In [73]:
uri = "mongodb://localhost:27017/"
client = MongoClient(uri)
try:
    database = client.get_database("GEO")
    data = database.get_collection("fuzhou")
    # Query
    query = { "properties.name": "闽侯县" }
    t = data.find(query)
    print(list(t))
    client.close()
except Exception as e:
    raise Exception("Unable to find the document due to the following error: ", e)

[{'_id': ObjectId('666468d8cc1a3fab89211c48'), 'type': 'Feature', 'properties': {'adcode': 350121, 'name': '闽侯县', 'center': [119.145117, 26.148567], 'centroid': [119.116333, 26.174897], 'childrenNum': 0, 'level': 'district', 'parent': {'adcode': 350100}, 'subFeatureIndex': 6, 'acroutes': [100000, 350000, 350100]}, 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[119.245534, 26.09422], [119.251637, 26.097152], [119.250122, 26.100097], [119.24786, 26.101756], [119.251808, 26.103581], [119.258104, 26.108185], [119.261476, 26.112209], [119.262223, 26.114532], [119.259406, 26.121223], [119.255372, 26.124651], [119.249845, 26.127416], [119.243549, 26.129199], [119.240262, 26.132268], [119.239345, 26.141944], [119.239772, 26.144957], [119.242076, 26.14862], [119.246195, 26.150901], [119.249631, 26.155213], [119.251595, 26.160216], [119.25168, 26.164583], [119.249162, 26.169654], [119.246601, 26.172045], [119.242183, 26.174049], [119.237467, 26.180502], [119.232259, 26.18093], [119.2270

创建集合

In [134]:
def create_collection(db, collection_name):
    # 显式创建集合
    if collection_name not in db.list_collection_names():
        db.create_collection(collection_name)
        print(f'Collection "{collection_name}" created.')
    else:
        print(f'Collection "{collection_name}" already exists.')

In [136]:
client = MongoClient('mongodb://localhost:27017')
db = client['GEO']
create_collection(db,'fzpoi')
client.close()

Collection "fzpoi" already exists.


通过插入操作导入POI数据

In [ ]:
client = MongoClient('mongodb://localhost:27017')
db = client['GEO']
collection = db['fzpoi']

# 读取json数据
with open("D:\GISData\FZ\POI\高德源数据\FZPOI.json") as f:
    geojson = json.load(f)

# 向数据库中插入数据
features = geojson['features']
collection.insert_many(features)

print('GeoJSON data has been split and stored successfully.')
client.close()

删除数据

In [143]:
client = MongoClient('mongodb://localhost:27017')
db = client['GEO']
test = db.get_collection(name='test')

In [151]:
# 插入数据
test.insert_many([{
      '_id': 'ObjectId("563237a41a4d68582c2509da")',
      'stock': "Brent Crude Futures",
      'qty': 250,
      'type': "buy-limit",
      'limit': 48.90,
      'creationts': 'ISODate("2015-11-01T12:30:15Z")',
      'expiryts': 'ISODate("2015-11-01T12:35:15Z")',
      'client': "Crude Traders Inc."
   },{
      '_id': 'ObjectId("563237a41a4d68582123frgrg")',
      'stock': "Brent Crude Futures",
      'qty': 250,
      'type': "buy-limit",
      'limit': 48.90,
      'creationts': 'ISODate("2015-11-01T12:30:15Z")',
      'expiryts': 'ISODate("2015-11-01T12:35:15Z")',
      'client': "Crude Traders Inc."
   }
   ])

InsertManyResult(['ObjectId("563237a41a4d68582c2509da")', 'ObjectId("563237a41a4d68582123frgrg")'], acknowledged=True)

In [152]:
# 删除数据
test.delete_one({ '_id': 'ObjectId("563237a41a4d68582c2509da")' },)

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

修改数据

In [153]:
test.update_one({ '_id': 'ObjectId("563237a41a4d68582123frgrg")'},{ '$set': { "qty" : 3 }} )

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [154]:
test.find_one()

{'_id': 'ObjectId("563237a41a4d68582123frgrg")',
 'stock': 'Brent Crude Futures',
 'qty': 3,
 'type': 'buy-limit',
 'limit': 48.9,
 'creationts': 'ISODate("2015-11-01T12:30:15Z")',
 'expiryts': 'ISODate("2015-11-01T12:35:15Z")',
 'client': 'Crude Traders Inc.'}

数据可视化

In [75]:
client = MongoClient('mongodb://localhost:27017')
db = client['GEO']
collection = db['fzpoi']

# 从 MongoDB 读取 GeoJSON 数据
features = list(collection.find())
for feature in features:
    if '_id' in feature:
        feature['_id'] = str(feature['_id'])
# 构建 GeoJSON 格式数据
geojson_data = {
    "type": "FeatureCollection",
    "features": features[0:100]
}

# 创建 Folium 地图对象 福州市
m = folium.Map(location=[26.0745, 119.2965], zoom_start=12)

# 添加 GeoJSON 图层
folium.GeoJson(geojson_data).add_to(m)

map_filename = 'fuzhoupoi_geojson_map.html'
m.save(map_filename)
print('GeoJSON data visualized')
client.close()
m

GeoJSON data visualized


创建地理索引

In [6]:
# 创建地理空间索引
client = MongoClient('mongodb://localhost:27017')
db = client['GEO']
collection = db['fzpoi']

# 创建地理空间索引
collection.create_index([("geometry", "2dsphere")])

print('Geospatial index has been created.')
client.close()


Geospatial index has been created.


空间查询

相交查询——查找poi所在的区域

In [7]:
# 连接数据库，读取数据集合
client = MongoClient('mongodb://localhost:27017')
db = client['GEO']
fz_collection = db['fuzhou']
poi_collection = db['fzpoi']

In [9]:
# 获取一个点坐标
point = poi_collection.find_one()
coordinates = point['geometry']['coordinates']

In [82]:
# 查询点所在的区域
poi_area = fz_collection.find_one({ 'geometry': { '$geoIntersects': {'$geometry': { 'type': "Point", 'coordinates': coordinates } } } })
poi_area['properties']['name']

'马尾区'

In [85]:
if '_id' in poi_area:
    poi_area['_id'] = str(poi_area['_id'])

In [93]:
# 创建 Folium 地图对象
m = folium.Map(location=[coordinates[1], coordinates[0]], zoom_start=12)

folium.Marker(
        location=[coordinates[1], coordinates[0]],
        icon=folium.Icon(icon='info-sign')
    ).add_to(m)

# 添加 GeoJSON 图层
folium.GeoJson(poi_area).add_to(m)
m

查找区域内的餐馆

In [43]:
area = fz_collection.find_one()
area_coordinates = area['geometry']['coordinates']

In [115]:
# 查询点所在的区域
area_poi = poi_collection.find({ 'geometry': { '$geoWithin': {'$geometry': { 'type': "MultiPolygon", 'coordinates': area_coordinates}} },'properties.小类':'中餐厅'})
area_poi

In [116]:
features = list(area_poi)
for feature in features:
    if '_id' in feature:
        feature['_id'] = str(feature['_id'])

if '_id' in area:
    area['_id'] = str(feature['_id'])

# 构建 GeoJSON 格式数据
geojson_data = {
    "type": "FeatureCollection",
    "features": features[:200]
}
# 统计数量
len(features)

1870

In [117]:
features

[{'_id': '66647127cc1a3fab892310e8',
  'type': 'Feature',
  'properties': {'name': '金牛饭店(杨桥西路店)',
   'cityname': '福州市',
   '大类': '餐饮服务',
   '中类': '中餐厅',
   '小类': '中餐厅',
   'lon': 119.256365,
   'lat': 26.077391,
   'wgs84_lon': 119.25160234706324,
   'wgs84_lat': 26.0805328306575},
  'geometry': {'type': 'Point',
   'coordinates': [119.25160234706324, 26.0805328306575]}},
 {'_id': '66647127cc1a3fab892310f0',
  'type': 'Feature',
  'properties': {'name': '五福记福鼎小吃(杨桥西路店)',
   'cityname': '福州市',
   '大类': '餐饮服务',
   '中类': '中餐厅',
   '小类': '中餐厅',
   'lon': 119.256508,
   'lat': 26.07677,
   'wgs84_lon': 119.25174536626136,
   'wgs84_lat': 26.07991138141688},
  'geometry': {'type': 'Point',
   'coordinates': [119.25174536626136, 26.07991138141688]}},
 {'_id': '66647128cc1a3fab8925a48e',
  'type': 'Feature',
  'properties': {'name': '曼玲粥店(杨桥店)',
   'cityname': '福州市',
   '大类': '餐饮服务',
   '中类': '中餐厅',
   '小类': '中餐厅',
   'lon': 119.256469,
   'lat': 26.076763,
   'wgs84_lon': 119.25170637428197,


In [118]:
# 创建 Folium 地图对象
m = folium.Map(location=[coordinates[1], coordinates[0]], zoom_start=12)
folium.GeoJson(area).add_to(m)
# 添加 GeoJSON 图层
folium.GeoJson(geojson_data).add_to(m)
m

查询周边的餐馆

In [119]:
poi_canguan = poi_collection.find({ 'geometry': 
                                    { '$near': 
                                     {
                                         '$geometry': { 'type': "point", 'coordinates': coordinates},
                                       '$minDistance': 1000,
                                       '$maxDistance': 5000
                                    }},
                                    'properties.大类':'餐饮服务'})
poi_canguan

In [120]:
features = list(poi_canguan)
for feature in features:
    if '_id' in feature:
        feature['_id'] = str(feature['_id'])

# 构建 GeoJSON 格式数据
geojson_data = {
    "type": "FeatureCollection",
    "features": features
}
# 统计数量
len(features)

111

In [123]:
# 创建 Folium 地图对象
m = folium.Map(location=[coordinates[1], coordinates[0]], zoom_start=12)

folium.Marker(
        location=[coordinates[1], coordinates[0]],
        icon=folium.Icon(icon='info-sign',color='red'),
    ).add_to(m)

# 添加 GeoJSON 图层
folium.GeoJson(geojson_data).add_to(m)
m

使用聚合查找点周边500m内的餐馆

In [129]:
pois = poi_collection.aggregate([
   {
     '$geoNear': {
        'near': { 'type': "Point", 'coordinates': coordinates },
        'distanceField': "dist.calculated",
        'maxDistance': 500,
        'query': { 'properties.大类': "餐饮服务" },
        'includeLocs': "dist.location",
        'spherical': 'true'
     }
   }
])

In [130]:
features = list(pois)
for feature in features:
    if '_id' in feature:
        feature['_id'] = str(feature['_id'])

# 构建 GeoJSON 格式数据
geojson_data = {
    "type": "FeatureCollection",
    "features": features
}
# 统计数量
len(features)

1

In [133]:
# 创建 Folium 地图对象
m = folium.Map(location=[coordinates[1], coordinates[0]], zoom_start=20)

folium.Marker(
        location=[coordinates[1], coordinates[0]],
        icon=folium.Icon(icon='info-sign',color='red'),
    ).add_to(m)

# 添加 GeoJSON 图层
folium.GeoJson(geojson_data).add_to(m)
m